In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Carregar os dados
dataset = pd.read_csv('../datasets/dataset.csv')

# Pré-Processamento dos Dados
dataset = pd.get_dummies(dataset, columns=['tipo', 'caixa', 'combustivel', 'marca', 'com_danos'])

# Lidar com possíveis valores nulos
dataset = dataset.dropna()

# Separar features (X) e target (y)
X = dataset.drop(['id', 'preco'], axis=1)
y = dataset['preco']

# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# Modelo Random Forest
rf_model = RandomForestRegressor(random_state=42)

# Treinar o modelo
rf_model.fit(X_train, y_train)

# Avaliar o modelo nos dados de teste
y_pred = rf_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2 Score on Test Data: {r2}')

R2 Score on Test Data: 0.752073853426942


In [6]:
# Estimar os preços para as viaturas com características no just_features.csv
just_features = pd.read_csv('../datasets/just_features.csv')

# Garantir que as features são as mesmas usadas para treino do modelo
just_features_processed = pd.get_dummies(just_features, columns=['tipo', 'caixa', 'combustivel', 'marca', 'com_danos'])
just_features_processed = just_features_processed.reindex(columns=X.columns, fill_value=0)

# Adicionar a coluna 'id' de volta ao just_features_processed
just_features_processed['id'] = just_features['id']

# Fazer as estimativas
estimated_prices = rf_model.predict(just_features_processed.drop('id', axis=1))

# Criar DataFrame com as estimativas
results_df = pd.DataFrame({'id': just_features_processed['id'], 'preco': estimated_prices})

# Salvar as estimativas em precos_estimados.csv
results_df.to_csv('precos_estimados.csv', index=False)